# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights


In [2]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

from __future__ import print_function

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Softmax Classifier

Your code for this section will all be written inside **cs231n/classifiers/softmax.py**. 


In [25]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs231n/classifiers/softmax.py and implement the
# softmax_loss_naive function.

from cs231n.classifiers.softmax import softmax_loss_naive
from cs231n.classifiers.softmax import softmax_loss_naive_soln
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('soln loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

loss: 2.364589
soln loss: 2.364589
sanity check: 2.302585


## Inline Question 1:
Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

**Your answer:** *Fill this in*
We expect all the scores to be evenly distributed, so e^S_yi = e^S_j for all j. This leads to the arguemnt to the
log function to be ~1/N where N is number of classes. In this case, N = 10. 

In [29]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive_soln(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive_soln(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

print("add regularization")
# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive_soln(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive_soln(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: 1.139897 analytic: 1.018378, relative error: 5.630401e-02
numerical: 0.480025 analytic: 0.397304, relative error: 9.428724e-02
numerical: 1.843396 analytic: 1.611966, relative error: 6.697709e-02
numerical: 1.602670 analytic: 1.408876, relative error: 6.435060e-02
numerical: 2.410375 analytic: 2.352210, relative error: 1.221275e-02
numerical: 1.778426 analytic: 1.750881, relative error: 7.804798e-03
numerical: -0.847236 analytic: -0.756367, relative error: 5.666610e-02
numerical: 0.167139 analytic: 0.114843, relative error: 1.854565e-01
numerical: 0.501509 analytic: 0.412622, relative error: 9.723618e-02
numerical: 3.789519 analytic: 3.362074, relative error: 5.976918e-02
add regularization
numerical: 1.031207 analytic: 0.957646, relative error: 3.698661e-02
numerical: 0.473369 analytic: 0.262744, relative error: 2.861307e-01
numerical: 1.734502 analytic: 1.504525, relative error: 7.100194e-02
numerical: 2.616403 analytic: 2.378853, relative error: 4.755522e-02
numerical: -0

In [30]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

print("add regularization")
# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: 2.184243 analytic: 2.184243, relative error: 1.665854e-09
numerical: -1.851608 analytic: -1.851608, relative error: 2.094729e-08
numerical: 3.647062 analytic: 3.647062, relative error: 9.040043e-09
numerical: -2.040290 analytic: -2.040290, relative error: 4.699818e-10
numerical: -1.803662 analytic: -1.803662, relative error: 4.695374e-09
numerical: 0.032086 analytic: 0.032086, relative error: 5.752071e-07
numerical: -1.231095 analytic: -1.231095, relative error: 7.840559e-08
numerical: -0.471769 analytic: -0.471769, relative error: 6.511260e-09
numerical: 0.210547 analytic: 0.210547, relative error: 4.383300e-09
numerical: -0.046948 analytic: -0.046948, relative error: 3.582322e-08
add regularization
numerical: 0.564775 analytic: 0.564775, relative error: 9.271522e-09
numerical: -0.747938 analytic: -0.747938, relative error: 1.641424e-08
numerical: 1.806685 analytic: 1.806685, relative error: 3.039931e-08
numerical: -0.083390 analytic: -0.083390, relative error: 2.026959e-07

In [46]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

from cs231n.classifiers.softmax import softmax_loss_vectorized
from cs231n.classifiers.softmax import softmax_loss_vectorized_soln

tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

naive loss: 2.366329e+00 computed in 0.266585s
S
(500, 10)
[[-0.48806417 -0.16424376  0.00749817 ...,  0.25395822  0.08129688
   0.34630356]
 [-0.50933788  0.46124392 -0.61024699 ...,  0.49642472  0.09487644
   0.10893162]
 [-0.83931913  0.50259181 -0.59721728 ...,  0.74869029 -0.19991291
   0.56288048]
 ..., 
 [ 0.6911079  -0.7301356   0.10397017 ..., -0.94460257 -0.15289159
  -0.15547923]
 [-0.03206701 -0.23722788 -0.09860362 ..., -0.22467863 -0.03093766
   0.21300962]
 [-0.36451755  0.32081483 -0.19879968 ...,  0.21101502 -0.06534451
   0.07764532]]
expS
(500, 10)
[[ 0.61381348  0.84853515  1.00752635 ...,  1.28911794  1.08469287
   1.41383174]
 [ 0.60089331  1.58604566  0.54321668 ...,  1.64283716  1.09952299
   1.11508609]
 [ 0.43200456  1.65299999  0.55034095 ...,  2.11422918  0.81880206
   1.75572256]
 ..., 
 [ 1.9959256   0.48184365  1.10956735 ...,  0.38883407  0.85822276
   0.85600485]
 [ 0.96844168  0.78881151  0.90610179 ...,  0.79877288  0.96953601
   1.23739655]
 [ 0.6945

In [44]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 5e-7]
regularization_strengths = [2.5e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
pass
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

best validation accuracy achieved during cross-validation: -1.000000


In [ ]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

In [ ]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
    
    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])